In [1]:
import os
import luigi
import io
import pandas as pd
import numpy as np
import logging
from emu.pipeline.download import FileManifest,PatientsLocal,CacheTaskOutput,ExperimentManifest
from emu.pipeline.remote import RemotePatientManifest,RemoteCSV,Patients
# from emu.pipeline.utils import file_ids_by_channel
from emu.luigi.box import BoxTarget, path_to_fid
from emu.pipeline.process import PDilTask
from emu.utils import get_file_manifest,load_patients,Experiment
from emu.auth import jwt
from emu.pdil.raw import download_experiment, BehaviorRaw, timestamps,Game,extract_trial_timing
from emu.pdil.raw import PDilCache as PDC
from emu.pdil.raw import Participant,get_data_manifest

In [2]:
root_dir = os.path.expanduser('~/.emu/')

In [3]:
pdc = PDC()

In [4]:
raw_files = pdc.data_manifest.query('patient_id == 1')

In [5]:
seeg_raw = os.path.join('/home/elijahc/.emu/pdil/pt_01/SEEG/raw')
p = Participant(patient_id=1,raw_files=raw_files,seeg_raw_path=seeg_raw)

In [6]:
games = p.load_game_data()
next(games)

DEBUG: Checking if BehaviorRaw(data_root=/home/elijahc/.emu/, file_id=629615908637, file_name=blockNum_3_computerTT_defect_taskoutput.mat, save_to=/home/elijahc/.emu/pdil/pt_01/Behavior/raw, overwrite=False, study=pdil, patient_id=1) is complete
INFO: Informed scheduler that task   BehaviorRaw__home_elijahc__e_629615908637_blockNum_3_compu_3484a4f2e7   has status   DONE
DEBUG: Checking if BehaviorRaw(data_root=/home/elijahc/.emu/, file_id=629620274728, file_name=blockNum_4_humanTT_defect_taskoutput.mat, save_to=/home/elijahc/.emu/pdil/pt_01/Behavior/raw, overwrite=False, study=pdil, patient_id=1) is complete
INFO: Informed scheduler that task   BehaviorRaw__home_elijahc__e_629620274728_blockNum_4_human_73d59337a6   has status   DONE
DEBUG: Checking if BehaviorRaw(data_root=/home/elijahc/.emu/, file_id=629611713944, file_name=blockNum_5_humanTT_coop_taskoutput.mat, save_to=/home/elijahc/.emu/pdil/pt_01/Behavior/raw, overwrite=False, study=pdil, patient_id=1) is complete
INFO: Informed s

,points,reaction_time,timing_sumTictocs,timing_wholesession,trial,block,opponent,strategy
0,4,1.887549,16.915350,16.937015,1,3,computer,defect
1,4,1.336412,14.620682,14.638466,2,3,computer,defect
2,6,1.206812,14.253415,14.264153,3,3,computer,defect
3,1,1.016557,14.099830,14.112599,4,3,computer,defect
4,6,3.820575,34.350554,34.375759,5,3,computer,defect
5,1,4.389647,20.046147,20.071950,6,3,computer,defect
6,1,1.622268,14.245598,14.263381,7,3,computer,defect
7,2,1.144656,14.924340,14.927988,8,3,computer,defect
8,2,1.290998,14.659205,14.671975,9,3,computer,defect
9,4,0.885510,32.880432,32.903222,10,3,computer,defect


In [7]:
list(p.cache_behavior())

[BehaviorRaw(data_root=/home/elijahc/.emu/, file_id=629615908637, file_name=blockNum_3_computerTT_defect_taskoutput.mat, save_to=/home/elijahc/.emu/pdil/pt_01/Behavior/raw, overwrite=False, study=pdil, patient_id=1),
 BehaviorRaw(data_root=/home/elijahc/.emu/, file_id=629620274728, file_name=blockNum_4_humanTT_defect_taskoutput.mat, save_to=/home/elijahc/.emu/pdil/pt_01/Behavior/raw, overwrite=False, study=pdil, patient_id=1),
 BehaviorRaw(data_root=/home/elijahc/.emu/, file_id=629611713944, file_name=blockNum_5_humanTT_coop_taskoutput.mat, save_to=/home/elijahc/.emu/pdil/pt_01/Behavior/raw, overwrite=False, study=pdil, patient_id=1),
 BehaviorRaw(data_root=/home/elijahc/.emu/, file_id=629611688205, file_name=blockNum_6_computerTT_coop_taskoutput.mat, save_to=/home/elijahc/.emu/pdil/pt_01/Behavior/raw, overwrite=False, study=pdil, patient_id=1),
 BehaviorRaw(data_root=/home/elijahc/.emu/, file_id=628721484958, file_name=blockNum_1_computerTT_coop_taskoutput.mat, save_to=/home/elijahc/.

In [8]:
pdil = pd.concat(p.load_pdil_events())
pdil.groupby(['block','trial']).count()

event  event_delta  screen
block trial                            
0     0          1            1       0
      1         10           10      10
      2         10           10      10
      3         10           10      10
      4         10           10      10
      5         10           10      10
      6         10           10      10
      7         10           10      10
      8         10           10      10
1     0          1            1       0
      1         10           10      10
      2         10           10      10
      3         10           10      10
      4         10           10      10
      5         10           10      10
      6         10           10      10
      7         10           10      10
      8         10           10      10
      9         10           10      10
      10        10           10      10
      11        10           10      10
      12        10           10      10
      13        10           10      10
      14        10           10      10
      15        10           10      10
2     0          1            1       0
      1         10           10      10
      2         10           10      10
      3         10           10      10
      4         10           10      10
...            ...          ...     ...
5     2         10           10      10
      3         10           10      10
      4         10           10      10
      5         10           10      10
      6         10           10      10
      7         10           10      10
      8         10           10      10
      9         10           10      10
      10        10           10      10
      11        10           10      10
      12        10           10      10
      13        10           10      10
      14        10           10      10
      15        10           10      10
6     0          1            1       0
      1         10           10      10
      2         10           10      10
      3         10           10      10
      4         10           10      10
      5         10           10      10
      6         10           10      10
      7         10           10      10
      8         10           10      10
      9         10           10      10
      10        10           10      10
      11        10           10      10
      12        10           10      10
      13        10           10      10
      14        10           10      10
      15        10           10      10

[105 rows x 3 columns]

In [ ]:
fp = '/home/elijahc/.emu/pdil/pt_01/Behavior/raw/blockNum_1_computerTT_coop_taskoutput.mat'
b1 = sio.loadmat(fp,squeeze_me=True,struct_as_record=False)

In [ ]:
to = b1['taskoutput']

In [ ]:
timing = b1['taskoutput'].timing

In [ ]:
timing.__dict__.keys()

In [ ]:
intro = [
    'Time_scrn_flip_intro1',
    'Time_postscrn_flip_intro1',
    'Time_scrn_flip_intro2',
    'Time_postscrn_flip_intro2',
    'Time_scrn_flip_intro3',
    'Time_postscrn_flip_intro3',
]

trial = [
    'Time_scrn_flip_trials1',
    'Time_postscrn_flip_trials1',
    'Time_scrn_flip_trials2',
    'Time_postscrn_flip_trials2',
    'Time_scrn_flip_trials3',
    'Time_postscrn_flip_trials3',
    'Time_scrn_flip_trials4',
    'Time_postscrn_flip_trials4',
    'Time_scrn_flip_trials5',
    'Time_postscrn_flip_trials5',
]

In [ ]:
num_rounds = 15
records = {
    'delta_t':[],
    'screen':[],
    'event':[],
    'round':[],
    'round_idx':[],
}

for i in np.arange(num_rounds):
    ri = 0
    for t in trial:
        ri+=1
        row = getattr(timing,t)
        records['delta_t'].append(row[i:i+1].max())
        records['round_idx'].append(ri)
        records['screen'].append(t[-1])
        if 'postscrn' in t:
            records['event'].append('keypress{}'.format(t[-1]))
        else:
            records['event'].append(t)
#         records['keypress'].append()
    records['round'].extend([i+1]*len(trial))
        

df = pd.DataFrame.from_records(records).sort_values(['round','screen'])
df['ttl_delta'] = df.delta_t.cumsum()
df.head(11)

In [ ]:
df = extract_trial_timing(fp)
df['ttl_delta'] = df.event_delta.cumsum()
df

In [ ]:
l = lambda s,j: s[:j].delta_t.sum()

In [ ]:
df.groupby('round').apply(l,8)

In [ ]:
getattr(timing,trial[7])

In [ ]:
game = pdc.load_behavior(patient_id=[1])
# game.head(20)